# Cargamos librerias y conectamos la base de datos

In [1]:
from pymongo import MongoClient
import time, json, random
from random import randrange
from datetime import timedelta, datetime
from random import randint
from lorem_text import lorem
from flask import jsonify

client = MongoClient('localhost', 27017)

In [2]:
db = client["test_mongo"]

# Generación de datos aleatorios

In [3]:
def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [4]:

d1 = datetime.strptime('1/1/1950 12:59 PM', '%m/%d/%Y %I:%M %p')
d2 = datetime.strptime('1/1/2022 01:00 AM', '%m/%d/%Y %I:%M %p')
print(random_date(d1, d2))  

1992-05-07 04:55:48


In [5]:
descripciones = [
    'Transferencia interbancaria SIMPLE',
    'DEB.CTA.P/C.INTERNET',
    'Debito Cta por ACH',
    'Retiro en efectivo',
    'AB.CTA.P/MOVIL',
    'Pago en interés',
    'Debito Cta por compra',
    'Debito Cta por pago de Seguro',
    'Retención IVA'
]

# Creación de índices

In [6]:
db.extracto_cuenta.create_index('fecha')
db.extracto_cuenta.create_index('agencia')
db.extracto_cuenta.create_index('monto')
db.extracto_cuenta.create_index('saldo')

'saldo_1'

In [7]:
db.extracto_cuenta.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'fecha_1': {'v': 2, 'key': [('fecha', 1)]},
 'agencia_1': {'v': 2, 'key': [('agencia', 1)]},
 'monto_1': {'v': 2, 'key': [('monto', 1)]},
 'saldo_1': {'v': 2, 'key': [('saldo', 1)]}}

# Aproximadamente 100 000 registros

## Carga de datos

### De forma individual

In [12]:
start_time = time.time()

for i in range(50000):
    fecha = random_date(d1, d2)
    agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
    monto = round(random.uniform(10.0, 4000.0), 2)
    descripcion = random.choice(descripciones)
    saldo = round(random.uniform(0.0, 10000000.0), 2)
    nota = lorem.words(randint(1,13))

    insert_data = {
        'fecha': fecha,
        'agencia': agencia,
        'monto': monto,
        'descripcion': descripcion,
        'saldo': saldo,
        'nota': nota
    }
    
    db.extracto_cuenta.insert_one(insert_data)


end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 0:00:53.038384


### De forma masiva

In [13]:
insert_data_list = []

for i in range(50000):
    fecha = random_date(d1, d2)
    agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
    monto = round(random.uniform(10.0, 4000.0), 2)
    descripcion = random.choice(descripciones)
    saldo = round(random.uniform(0.0, 10000000.0), 2)
    nota = lorem.words(randint(1,13))

    insert_data = {
        'fecha': fecha,
        'agencia': agencia,
        'monto': monto,
        'descripcion': descripcion,
        'saldo': saldo,
        'nota': nota
    }

    insert_data_list.append(insert_data)
start_time = time.time()
db.extracto_cuenta.insert_many(insert_data_list)
end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 0:00:02.658961


### Un solo dato

In [14]:
start_time = time.time()

fecha = random_date(d1, d2)
agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
monto = round(random.uniform(10.0, 4000.0), 2)
descripcion = random.choice(descripciones)
saldo = round(random.uniform(0.0, 10000000.0), 2)
nota = lorem.words(randint(1,13))

insert_data = {
    'fecha': fecha,
    'agencia': agencia,
    'monto': monto,
    'descripcion': descripcion,
    'saldo': saldo,
    'nota': nota
}

db.extracto_cuenta.insert_one(insert_data)


end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 0:00:00.006146


In [15]:
db.extracto_cuenta.count_documents({})

100001

## Consultas a la base de datos

### Búsqueda por fecha y agencia

In [16]:
def busqueda_fecha_agencia(month, year, agencia):
    from_date_str = f"{year}-{month}-01" + "T00:00:00+00:00"
    to_date_str = f"{year}-{month}-30" + "T23:59:59+00:00"
    from_date = datetime.strptime(from_date_str, '%Y-%m-%dT%H:%M:%S%z')
    to_date = datetime.strptime(to_date_str, '%Y-%m-%dT%H:%M:%S%z')
    criteria = {"$and": [{"fecha": {"$gte": from_date, "$lte": to_date}}, {"agencia": f"{agencia}"}]}

    return db.extracto_cuenta.find(criteria)

In [17]:
start_time = time.time()
results = busqueda_fecha_agencia(month=5, year = 2021, agencia = "La Paz")
end_time = time.time()

for result in results[:10]:
    print(result)
    

print("="*300)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print(f"Llaves examinadas {results.explain()['executionStats']['totalKeysExamined']}")
print(f"Documentos examinados {results.explain()['executionStats']['totalDocsExamined']}")


{'_id': ObjectId('6362f247dfb1bb16c3f47e1a'), 'fecha': datetime.datetime(2021, 5, 4, 3, 11, 1), 'agencia': 'La Paz', 'monto': 3637.26, 'descripcion': 'AB.CTA.P/MOVIL', 'saldo': 5861063.82, 'nota': 'neque accusamus saepe'}
{'_id': ObjectId('6362f247dfb1bb16c3f43b19'), 'fecha': datetime.datetime(2021, 5, 9, 12, 59, 24), 'agencia': 'La Paz', 'monto': 1541.18, 'descripcion': 'Debito Cta por pago de Seguro', 'saldo': 3735523.28, 'nota': 'corporis necessitatibus praesentium ullam fugit soluta deserunt vel consequuntur odit voluptate reiciendis nihil'}
{'_id': ObjectId('6362f21adfb1bb16c3f32872'), 'fecha': datetime.datetime(2021, 5, 9, 23, 35), 'agencia': 'La Paz', 'monto': 577.27, 'descripcion': 'Debito Cta por ACH', 'saldo': 2133135.05, 'nota': 'placeat nostrum doloremque eius vel dolores corporis at'}
{'_id': ObjectId('6362f231dfb1bb16c3f38a8b'), 'fecha': datetime.datetime(2021, 5, 12, 21, 28), 'agencia': 'La Paz', 'monto': 2444.37, 'descripcion': 'Pago en interés', 'saldo': 2655960.53, 'n

### Búsqueda por montos mayores a determinado número y agencia

In [18]:
def busqueda_monto_mayor(monto, agencia):
    criteria = {"$and": [{"monto": {"$gte": monto}}, {"agencia": agencia}]}

    return db.extracto_cuenta.find(criteria)

In [19]:
start_time = time.time()
results =  busqueda_monto_mayor(monto=3000, agencia = "Cochabamba")
end_time = time.time()

for result in results[:10]:
    print(result)
    
#print(f"Se encontraron {results.count()} registros")
print("="*300)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print(f"Llaves examinadas {results.explain()['executionStats']['totalKeysExamined']}")
print(f"Documentos examinados {results.explain()['executionStats']['totalDocsExamined']}")


{'_id': ObjectId('6362f20fdfb1bb16c3f303ac'), 'fecha': datetime.datetime(2005, 4, 17, 19, 14, 35), 'agencia': 'Cochabamba', 'monto': 3206.88, 'descripcion': 'Transferencia interbancaria SIMPLE', 'saldo': 8784823.11, 'nota': 'ratione corporis sint labore sapiente fuga'}
{'_id': ObjectId('6362f210dfb1bb16c3f3042b'), 'fecha': datetime.datetime(1968, 3, 20, 22, 59, 17), 'agencia': 'Cochabamba', 'monto': 3955.64, 'descripcion': 'Debito Cta por pago de Seguro', 'saldo': 6925027.44, 'nota': 'eos'}
{'_id': ObjectId('6362f210dfb1bb16c3f3044d'), 'fecha': datetime.datetime(1984, 6, 2, 16, 33, 40), 'agencia': 'Cochabamba', 'monto': 3035.53, 'descripcion': 'Retiro en efectivo', 'saldo': 8791310.64, 'nota': 'a nam aliquid'}
{'_id': ObjectId('6362f210dfb1bb16c3f30463'), 'fecha': datetime.datetime(1997, 4, 17, 9, 10, 22), 'agencia': 'Cochabamba', 'monto': 3882.39, 'descripcion': 'Debito Cta por compra', 'saldo': 3890252.57, 'nota': 'soluta molestias architecto quam voluptate illo'}
{'_id': ObjectId('6

### Búsqueda considerando varios campos

In [20]:
def busqueda_compleja(saldo_min, saldo_max, agencia1, agencia2, descripcion):
    criteria = {
        "$and": [
            {"$or": [{"agencia": agencia1}, {"agencia": agencia2}]},
            {"saldo": {"$gte": saldo_min, "$lte": saldo_max}},
            {"descripcion": descripcion}
        ]}

    return db.extracto_cuenta.find(criteria)

In [21]:
start_time = time.time()
results = busqueda_compleja(saldo_min=1000, saldo_max=100000000, agencia1="Cochabamba", agencia2="Santa Cruz", descripcion="DEB.CTA.P/C.INTERNET")
end_time = time.time()

for result in results[:10]:
    print(result)
    
#print(f"Se encontraron {results.count()} registros")
print("="*300)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print(f"Llaves examinadas {results.explain()['executionStats']['totalKeysExamined']}")
print(f"Documentos examinados {results.explain()['executionStats']['totalDocsExamined']}")


{'_id': ObjectId('6362f210dfb1bb16c3f30432'), 'fecha': datetime.datetime(1988, 5, 9, 10, 51, 15), 'agencia': 'Cochabamba', 'monto': 1422.29, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 7991765.85, 'nota': 'nobis illo'}
{'_id': ObjectId('6362f210dfb1bb16c3f304e4'), 'fecha': datetime.datetime(1958, 6, 8, 23, 57, 4), 'agencia': 'Cochabamba', 'monto': 2835.41, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 7343992.88, 'nota': 'libero necessitatibus iste laborum non eius sed perferendis'}
{'_id': ObjectId('6362f210dfb1bb16c3f3056f'), 'fecha': datetime.datetime(1961, 3, 16, 5, 52, 36), 'agencia': 'Cochabamba', 'monto': 2402.72, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 7006187.04, 'nota': 'praesentium rem numquam mollitia'}
{'_id': ObjectId('6362f210dfb1bb16c3f305fb'), 'fecha': datetime.datetime(2021, 10, 28, 17, 54, 44), 'agencia': 'Cochabamba', 'monto': 3515.16, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 636238.03, 'nota': 'possimus reiciendis'}
{'_id': ObjectId('6362f210df

### Agregaciones: Monto promedio por agencia

In [22]:
def promedio_monto_agencia(agencia):
    criteria = [
        {
        "$group" : 
            {"_id" : {"agencia" : f"{agencia}"},  
            "Monto promedio" : {"$avg" : "$monto"}
            }
        }
     ]

    return db.extracto_cuenta.aggregate(criteria)

In [23]:
start_time = time.time()
results =  promedio_monto_agencia(agencia="La Paz")
end_time = time.time()

for i in results:
    print(i)
    
#print(f"Se encontraron {results.count()} registros")
print("="*300)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

{'_id': {'agencia': 'La Paz'}, 'Monto promedio': 2003.2067002329975}
En un tiempo de 0:00:00.295489


## Actualización de datos

In [24]:
def actualizacion_fecha_monto(min_date, max_date, monto):
    from_date_str = min_date + "T00:00:00+00:00"
    to_date_str = max_date + "T23:59:59+00:00"
    from_date = datetime.strptime(from_date_str, '%Y-%m-%dT%H:%M:%S%z')
    to_date = datetime.strptime(to_date_str, '%Y-%m-%dT%H:%M:%S%z')
    
    criteria = {"$and": [{"fecha": {"$gte": from_date, "$lte": to_date}}]}
    new = {"$set": { "monto" : monto, "nota": "Monto actualizado por fecha"}}

    return db.extracto_cuenta.update_many(criteria, new)

In [25]:
from_date_str = "2021-01-01" + "T00:00:00+00:00"
to_date_str = "2021-01-31" + "T23:59:59+00:00"
from_date = datetime.strptime(from_date_str, '%Y-%m-%dT%H:%M:%S%z')
to_date = datetime.strptime(to_date_str, '%Y-%m-%dT%H:%M:%S%z')
db.extracto_cuenta.count_documents({"$and": [{"fecha": {"$gte": from_date}}, {"fecha": {"$lte": to_date}}]})

136

In [26]:
start_time = time.time()
results = actualizacion_fecha_monto(min_date = "2021-01-01", max_date = "2021-01-31", monto = 7000)
end_time = time.time()


print(results.modified_count)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

136
En un tiempo de 0:00:00.022670


In [27]:
results = db.extracto_cuenta.find({"$and": [{"fecha": {"$gte": from_date}}, {"fecha": {"$lte": to_date}}]})

for result in results[:10]:
    print(result)

{'_id': ObjectId('6362f247dfb1bb16c3f4858d'), 'fecha': datetime.datetime(2021, 1, 1, 0, 34, 4), 'agencia': 'Tarija', 'monto': 7000, 'descripcion': 'Debito Cta por ACH', 'saldo': 4109593.14, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('6362f247dfb1bb16c3f42212'), 'fecha': datetime.datetime(2021, 1, 1, 8, 19, 58), 'agencia': 'Cochabamba', 'monto': 7000, 'descripcion': 'Retención IVA', 'saldo': 5930967.31, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('6362f246dfb1bb16c3f4059c'), 'fecha': datetime.datetime(2021, 1, 1, 14, 11, 20), 'agencia': 'Tarija', 'monto': 7000, 'descripcion': 'Debito Cta por ACH', 'saldo': 1329299.07, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('6362f22ddfb1bb16c3f37f7c'), 'fecha': datetime.datetime(2021, 1, 1, 16, 15, 46), 'agencia': 'Tarija', 'monto': 7000, 'descripcion': 'Transferencia interbancaria SIMPLE', 'saldo': 1371073.32, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('6362f247dfb1bb16c3f46e3b'), 'fecha': datetime.

## Eliminar datos

In [28]:
def eliminar_saldo_min(saldo_min):
    return db.extracto_cuenta.delete_many(
        {"saldo": {"$lte": saldo_min}}
)

In [32]:
start_time = time.time()
results = eliminar_saldo_min(saldo_min= 500)
end_time = time.time()

print(results.deleted_count)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

2
En un tiempo de 0:00:00.002253


In [35]:
results = db.extracto_cuenta.count_documents({"saldo": {"$lte": 500}})
print(results)

0


# Más de 10 millones de documentos

## Carga de datos

In [36]:
insert_data_list = []

for _ in range(10000000):
    fecha = random_date(d1, d2)
    agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
    monto = round(random.uniform(10.0, 4000.0), 2)
    descripcion = random.choice(descripciones)
    saldo = round(random.uniform(0.0, 10000000.0), 2)
    nota = lorem.words(randint(1,13))

    insert_data = {
        'fecha': fecha,
        'agencia': agencia,
        'monto': monto,
        'descripcion': descripcion,
        'saldo': saldo,
        'nota': nota
    }

    insert_data_list.append(insert_data)

start_time = time.time()
db.extracto_cuenta.insert_many(insert_data_list)
end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 0:14:02.773019


In [41]:
db.extracto_cuenta.count_documents({})

10099999

## Consultas a la base de datos

### Búsqueda por fecha y agencia

In [42]:
start_time = time.time()
results = busqueda_fecha_agencia(month=5, year = 2021, agencia = "La Paz")
end_time = time.time()

for result in results[:10]:
    print(result)
    

print("="*300)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print(f"Llaves examinadas {results.explain()['executionStats']['totalKeysExamined']}")
print(f"Documentos examinados {results.explain()['executionStats']['totalDocsExamined']}")


{'_id': ObjectId('6362f408dfb1bb16c30085db'), 'fecha': datetime.datetime(2021, 5, 1, 0, 32, 8), 'agencia': 'La Paz', 'monto': 1593.08, 'descripcion': 'Pago en interés', 'saldo': 1748212.41, 'nota': 'culpa nesciunt ullam voluptatem iusto ut pariatur est soluta'}
{'_id': ObjectId('6362f46fdfb1bb16c36e2ca1'), 'fecha': datetime.datetime(2021, 5, 1, 0, 45, 18), 'agencia': 'La Paz', 'monto': 3120.46, 'descripcion': 'Retención IVA', 'saldo': 767001.15, 'nota': 'est maxime voluptatum'}
{'_id': ObjectId('6362f48adfb1bb16c38a9d4d'), 'fecha': datetime.datetime(2021, 5, 1, 1, 42, 49), 'agencia': 'La Paz', 'monto': 2430.24, 'descripcion': 'Debito Cta por pago de Seguro', 'saldo': 6926855.31, 'nota': 'fuga assumenda minus beatae explicabo'}
{'_id': ObjectId('6362f44cdfb1bb16c34bafcf'), 'fecha': datetime.datetime(2021, 5, 1, 2, 33, 53), 'agencia': 'La Paz', 'monto': 3719.4, 'descripcion': 'Debito Cta por ACH', 'saldo': 8994399.68, 'nota': 'veritatis impedit rem necessitatibus quaerat minus unde place

### Búsqueda por montos mayores a determinado número y agencia

In [43]:
start_time = time.time()
results =  busqueda_monto_mayor(monto=3000, agencia = "Cochabamba")
end_time = time.time()

for result in results[:10]:
    print(result)
    
#print(f"Se encontraron {results.count()} registros")
print("="*300)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print(f"Llaves examinadas {results.explain()['executionStats']['totalKeysExamined']}")
print(f"Documentos examinados {results.explain()['executionStats']['totalDocsExamined']}")


{'_id': ObjectId('6362f20fdfb1bb16c3f303ac'), 'fecha': datetime.datetime(2005, 4, 17, 19, 14, 35), 'agencia': 'Cochabamba', 'monto': 3206.88, 'descripcion': 'Transferencia interbancaria SIMPLE', 'saldo': 8784823.11, 'nota': 'ratione corporis sint labore sapiente fuga'}
{'_id': ObjectId('6362f210dfb1bb16c3f3042b'), 'fecha': datetime.datetime(1968, 3, 20, 22, 59, 17), 'agencia': 'Cochabamba', 'monto': 3955.64, 'descripcion': 'Debito Cta por pago de Seguro', 'saldo': 6925027.44, 'nota': 'eos'}
{'_id': ObjectId('6362f210dfb1bb16c3f3044d'), 'fecha': datetime.datetime(1984, 6, 2, 16, 33, 40), 'agencia': 'Cochabamba', 'monto': 3035.53, 'descripcion': 'Retiro en efectivo', 'saldo': 8791310.64, 'nota': 'a nam aliquid'}
{'_id': ObjectId('6362f210dfb1bb16c3f30463'), 'fecha': datetime.datetime(1997, 4, 17, 9, 10, 22), 'agencia': 'Cochabamba', 'monto': 3882.39, 'descripcion': 'Debito Cta por compra', 'saldo': 3890252.57, 'nota': 'soluta molestias architecto quam voluptate illo'}
{'_id': ObjectId('6

### Búsqueda considerando varios campos

In [44]:
start_time = time.time()
results = busqueda_compleja(saldo_min=1000, saldo_max=100000000, agencia1="Cochabamba", agencia2="Santa Cruz", descripcion="DEB.CTA.P/C.INTERNET")
end_time = time.time()

for result in results[:10]:
    print(result)
    
#print(f"Se encontraron {results.count()} registros")
print("="*300)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print(f"Llaves examinadas {results.explain()['executionStats']['totalKeysExamined']}")
print(f"Documentos examinados {results.explain()['executionStats']['totalDocsExamined']}")


{'_id': ObjectId('6362f210dfb1bb16c3f30432'), 'fecha': datetime.datetime(1988, 5, 9, 10, 51, 15), 'agencia': 'Cochabamba', 'monto': 1422.29, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 7991765.85, 'nota': 'nobis illo'}
{'_id': ObjectId('6362f210dfb1bb16c3f304e4'), 'fecha': datetime.datetime(1958, 6, 8, 23, 57, 4), 'agencia': 'Cochabamba', 'monto': 2835.41, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 7343992.88, 'nota': 'libero necessitatibus iste laborum non eius sed perferendis'}
{'_id': ObjectId('6362f210dfb1bb16c3f3056f'), 'fecha': datetime.datetime(1961, 3, 16, 5, 52, 36), 'agencia': 'Cochabamba', 'monto': 2402.72, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 7006187.04, 'nota': 'praesentium rem numquam mollitia'}
{'_id': ObjectId('6362f210dfb1bb16c3f305fb'), 'fecha': datetime.datetime(2021, 10, 28, 17, 54, 44), 'agencia': 'Cochabamba', 'monto': 3515.16, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 636238.03, 'nota': 'possimus reiciendis'}
{'_id': ObjectId('6362f210df

### Agregaciones: Monto promedio por agencia

In [45]:
start_time = time.time()
results =  promedio_monto_agencia(agencia="La Paz")
end_time = time.time()

for i in results:
    print(i)
    
#print(f"Se encontraron {results.count()} registros")
print("="*300)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

{'_id': {'agencia': 'La Paz'}, 'Monto promedio': 2005.150881605038}
En un tiempo de 0:00:19.013102


## Actualización de datos

In [46]:
from_date_str = "2021-01-01" + "T00:00:00+00:00"
to_date_str = "2021-01-31" + "T23:59:59+00:00"
from_date = datetime.strptime(from_date_str, '%Y-%m-%dT%H:%M:%S%z')
to_date = datetime.strptime(to_date_str, '%Y-%m-%dT%H:%M:%S%z')
db.extracto_cuenta.count_documents({"$and": [{"fecha": {"$gte": from_date}}, {"fecha": {"$lte": to_date}}]})

11864

In [47]:
start_time = time.time()
results = actualizacion_fecha_monto(min_date = "2021-01-01", max_date = "2021-01-31", monto = 7000)
end_time = time.time()


print(results.modified_count)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

11728
En un tiempo de 0:00:10.023878


In [48]:
results = db.extracto_cuenta.find({"$and": [{"fecha": {"$gte": from_date}}, {"fecha": {"$lte": to_date}}]})

for result in results[:10]:
    print(result)

{'_id': ObjectId('6362f447dfb1bb16c343ae80'), 'fecha': datetime.datetime(2021, 1, 1, 0, 1, 11), 'agencia': 'Cochabamba', 'monto': 7000, 'descripcion': 'Transferencia interbancaria SIMPLE', 'saldo': 18937.99, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('6362f425dfb1bb16c31f814e'), 'fecha': datetime.datetime(2021, 1, 1, 0, 2, 46), 'agencia': 'Cochabamba', 'monto': 7000, 'descripcion': 'Pago en interés', 'saldo': 8095910.46, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('6362f475dfb1bb16c376ed82'), 'fecha': datetime.datetime(2021, 1, 1, 0, 14, 48), 'agencia': 'Santa Cruz', 'monto': 7000, 'descripcion': 'Debito Cta por ACH', 'saldo': 9102110.08, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('6362f458dfb1bb16c35648f9'), 'fecha': datetime.datetime(2021, 1, 1, 0, 16, 57), 'agencia': 'Beni', 'monto': 7000, 'descripcion': 'Debito Cta por ACH', 'saldo': 3609916.16, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('6362f3fcdfb1bb16c3f7241e'), 'fecha': dateti

## Eliminar datos

In [49]:
start_time = time.time()
results = eliminar_saldo_min(saldo_min= 500)
end_time = time.time()

print(results.deleted_count)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

480
En un tiempo de 0:00:12.196713


In [50]:
results = db.extracto_cuenta.count_documents({"saldo": {"$lte": 500}})
print(results)

0
